In [ ]:
import os
import glob
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# --- Rutas base (la notebook está en por_muestras) ---
BASE_DIR = Path(".").resolve()

# Carpeta donde están los logs XLSX
RUTA_LOGS = BASE_DIR

# Carpeta de salida para el reporte
OUTPUT_DIR = BASE_DIR / "report"
IMG_DIR = OUTPUT_DIR / "img"

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
IMG_DIR.mkdir(parents=True, exist_ok=True)

print("BASE_DIR   :", BASE_DIR)
print("RUTA_LOGS  :", RUTA_LOGS)
print("OUTPUT_DIR :", OUTPUT_DIR)
print("IMG_DIR    :", IMG_DIR)


In [ ]:
def parsear_nombre_log(path_xlsx: Path) -> dict:
    """
    Recibe algo tipo:
      log_pcsmote_x_muestra_us_crime_D25_R25_PPentropia_I5.xlsx

    Devuelve un diccionario con:
      dataset, D, R, P_raw, P, I, base_tag
    """
    nombre = path_xlsx.name
    nombre_sin_ext = nombre.replace(".xlsx", "")

    prefijo = "log_pcsmote_x_muestra_"
    if nombre_sin_ext.startswith(prefijo):
        nombre_core = nombre_sin_ext[len(prefijo):]
    else:
        nombre_core = nombre_sin_ext

    partes = nombre_core.split("_")

    if len(partes) < 6:
        raise ValueError(f"No se puede parsear el nombre: {nombre}")

    # ..._Dxx_Ryy_Palgo_Iz
    dens_token = partes[-4]
    ries_token = partes[-3]
    pureza_token = partes[-2]
    limpieza_token = partes[-1]
    dataset = "_".join(partes[:-4])

    # D25 -> 25, R50 -> 50, I5 -> 5
    D = dens_token[1:] if dens_token.startswith("D") else dens_token
    R = ries_token[1:] if ries_token.startswith("R") else ries_token
    I = limpieza_token[1:] if limpieza_token.startswith("I") else limpieza_token

    # Pproporcion / PPentropia / Pentropia -> proporcion / entropia
    P_raw = pureza_token
    P = pureza_token.lstrip("P").lower()

    base_tag = f"{dataset}_D{D}_R{R}_P{P}_I{I}"

    return {
        "dataset": dataset,
        "D": D,
        "R": R,
        "P_raw": P_raw,
        "P": P,
        "I": I,
        "base_tag": base_tag,
        "nombre_archivo": nombre,
    }


# Prueba rápida con un archivo real (si existe alguno)
ejemplos = sorted(RUTA_LOGS.glob("log_pcsmote_x_muestra_*.xlsx"))
if ejemplos:
    print("Ejemplo parseo:", parsear_nombre_log(ejemplos[0]))
else:
    print("No se encontraron XLSX todavía.")


In [ ]:
logs = []

for path in sorted(RUTA_LOGS.glob("log_pcsmote_x_muestra_*.xlsx")):
    meta = parsear_nombre_log(path)
    print("📄 Cargando:", meta["nombre_archivo"])
    df = pd.read_excel(path)

    # Anoto metadatos dentro del dataframe (por si después querés concatenar)
    df["__dataset__"] = meta["dataset"]
    df["__D__"] = meta["D"]
    df["__R__"] = meta["R"]
    df["__P__"] = meta["P"]
    df["__I__"] = meta["I"]

    logs.append({"path": path, "meta": meta, "df": df})

print(f"\nTotal logs cargados: {len(logs)}")


In [ ]:
def hist_columna(ax, df, col, titulo, xlabel, ylabel="Número de semillas", bins=30):
    """
    Histograma genérico:
      - eje X: valores de la métrica (ej. riesgo, entropía, etc.)
      - eje Y: cantidad de semillas que tienen ese valor (frecuencia)
    """
    if col in df.columns:
        serie = df[col].dropna()
        if len(serie) > 0:
            ax.hist(serie, bins=bins)
        ax.set_title(titulo)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
    else:
        ax.text(0.5, 0.5, f"Sin columna\n{col}", ha="center", va="center")
        ax.set_xticks([])
        ax.set_yticks([])


def scatter_columna(ax, df, x_col, y_col, titulo,
                    xlabel=None, ylabel=None):
    """
    Scatter genérico:
      - eje X: métrica de PCSMOTE (riesgo, entropía, etc.)
      - eje Y: normalmente 'synthetics_from_this_seed'
    """
    if x_col in df.columns and y_col in df.columns:
        x = df[x_col].values
        y = df[y_col].values
        ax.scatter(x, y, s=10, alpha=0.5)
        ax.set_title(titulo)
        ax.set_xlabel(xlabel if xlabel is not None else x_col)
        ax.set_ylabel(ylabel if ylabel is not None else y_col)
    else:
        ax.text(0.5, 0.5, f"Sin columnas\n{x_col} / {y_col}", ha="center", va="center")
        ax.set_xticks([])
        ax.set_yticks([])


In [ ]:
# =====================================================================
# FIGURA 1 – DISTRIBUCIONES UNIVARIADAS
# FIGURA 2 – SINTÉTICOS vs MÉTRICAS
# FIGURA 3 – RELACIONES ENTRE MÉTRICAS
# FIGURA 4 – RIESGO vs ENTROPÍA coloreado por sintéticos
# FIGURA 5 – SINTÉTICOS EXTREMOS (outliers de generación)
# FIGURA 6 – “FRONTERA IDEAL” según percentiles (riesgo/entropía/densidad)
# =====================================================================

def generar_figura_distribuciones(df, meta, output_dir: Path) -> str:
    """
    Fig 1: distribuciones univariadas (6 subplots).

    Cada histograma muestra:
      - eje X: valor de la métrica calculada por PCSMOTE para cada semilla minoritaria
      - eje Y: número de semillas que tienen ese valor.
    """
    titulo_base = (
        f"{meta['dataset']}  D={meta['D']}  R={meta['R']}  "
        f"P={meta['P']}  I={meta['I']}"
    )

    fig, axes = plt.subplots(2, 3, figsize=(16, 9))
    axes = axes.ravel()

    # 1) Sintéticos por semilla
    hist_columna(
        axes[0], df,
        col="synthetics_from_this_seed",
        titulo="Sintéticos por semilla",
        xlabel="Cantidad de sintéticos generados por cada semilla",
        ylabel="Número de semillas"
    )

    # 2) Riesgo = proporción de vecinos mayoritarios (y=clase minoritaria)
    hist_columna(
        axes[1], df,
        col="riesgo",
        titulo="Riesgo",
        xlabel="Proporción de vecinos mayoritarios (riesgo)",
        ylabel="Número de semillas"
    )

    # 3) Entropía del vecindario de clases
    hist_columna(
        axes[2], df,
        col="entropia",
        titulo="Entropía",
        xlabel="Entropía de clases en el vecindario (0 = puro, alto = mezclado)",
        ylabel="Número de semillas"
    )

    # 4) Densidad local minoritaria
    hist_columna(
        axes[3], df,
        col="densidad",
        titulo="Densidad",
        xlabel="Densidad relativa en el vecindario minoritario",
        ylabel="Número de semillas"
    )

    # 5) Proporción de vecinos minoritarios
    hist_columna(
        axes[4], df,
        col="proporcion_min",
        titulo="Proporción minoritaria",
        xlabel="Proporción de vecinos minoritarios en el vecindario global",
        ylabel="Número de semillas"
    )

    # 6) Semillas usadas / descartadas por PCSMOTE
    ax = axes[5]
    if "is_filtrada" in df.columns:
        counts = df["is_filtrada"].value_counts()
        usadas = counts.get(True, 0)
        descartadas = counts.get(False, 0)

        ax.bar(["Usadas por PCSMOTE", "Descartadas por PCSMOTE"],
               [usadas, descartadas])

        ax.set_title("Semillas usadas / descartadas por PCSMOTE")
        ax.set_xlabel("Estado dentro de PCSMOTE")
        ax.set_ylabel("Número de semillas")
    else:
        ax.text(0.5, 0.5, "Sin columna is_filtrada", ha="center", va="center")
        ax.set_xticks([])
        ax.set_yticks([])

    fig.suptitle(
        "Distribuciones por semilla – "
        + titulo_base
        + "\nHist: eje X = valor de la métrica; eje Y = número de semillas. "
          "Último gráfico: semillas usadas/descartadas por PCSMOTE (no por IsolationCleaner).",
        fontsize=13
    )

    fig.tight_layout(rect=[0, 0, 1, 0.90])

    fname = f"{meta['base_tag']}_fig1_distribuciones.png"
    path = output_dir / fname
    fig.savefig(path, dpi=150)
    plt.close(fig)
    return fname


def generar_figura_vs_sinteticos(df, meta, output_dir: Path) -> str:
    """
    Fig 2: relación entre sintéticos y métricas (6 subplots).
    """
    titulo_base = (
        f"{meta['dataset']}  D={meta['D']}  R={meta['R']}  "
        f"P={meta['P']}  I={meta['I']}"
    )

    fig, axes = plt.subplots(2, 3, figsize=(16, 9))
    axes = axes.ravel()

    y_col = "synthetics_from_this_seed"

    scatter_columna(axes[0], df, "riesgo", y_col,
                    "Sintéticos vs Riesgo",
                    xlabel="Riesgo (vecinos mayoritarios)",
                    ylabel="Sintéticos generados")

    scatter_columna(axes[1], df, "entropia", y_col,
                    "Sintéticos vs Entropía",
                    xlabel="Entropía vecindario",
                    ylabel="Sintéticos generados")

    scatter_columna(axes[2], df, "densidad", y_col,
                    "Sintéticos vs Densidad",
                    xlabel="Densidad minoritaria",
                    ylabel="Sintéticos generados")

    scatter_columna(axes[3], df, "proporcion_min", y_col,
                    "Sintéticos vs Proporción minoritaria",
                    xlabel="Proporción vecinos minoritarios",
                    ylabel="Sintéticos generados")

    scatter_columna(axes[4], df, "vecinos_validos_por_percentil", y_col,
                    "Sintéticos vs Vecinos válidos",
                    xlabel="Vecinos válidos tras corte por percentil",
                    ylabel="Sintéticos generados")

    scatter_columna(axes[5], df, "percentil_dist_75", y_col,
                    "Sintéticos vs Distancia percentil",
                    xlabel="Umbral de distancia (percentil)",
                    ylabel="Sintéticos generados")

    fig.suptitle(
        "Sintéticos vs métricas de vecindario – " + titulo_base,
        fontsize=13
    )
    fig.tight_layout(rect=[0, 0, 1, 0.93])

    fname = f"{meta['base_tag']}_fig2_vs_sinteticos.png"
    path = output_dir / fname
    fig.savefig(path, dpi=150)
    plt.close(fig)
    return fname


def generar_figura_relaciones_internas(df, meta, output_dir: Path) -> str:
    """
    Fig 3: relaciones entre riesgo, densidad, entropía y proporción (6 subplots).
    """
    titulo_base = (
        f"{meta['dataset']}  D={meta['D']}  R={meta['R']}  "
        f"P={meta['P']}  I={meta['I']}"
    )

    fig, axes = plt.subplots(2, 3, figsize=(16, 9))
    axes = axes.ravel()

    scatter_columna(axes[0], df, "riesgo", "entropia",
                    "Riesgo vs Entropía",
                    xlabel="Riesgo", ylabel="Entropía")

    scatter_columna(axes[1], df, "riesgo", "densidad",
                    "Riesgo vs Densidad",
                    xlabel="Riesgo", ylabel="Densidad")

    scatter_columna(axes[2], df, "densidad", "entropia",
                    "Densidad vs Entropía",
                    xlabel="Densidad", ylabel="Entropía")

    scatter_columna(axes[3], df, "riesgo", "proporcion_min",
                    "Riesgo vs Proporción minoritaria",
                    xlabel="Riesgo", ylabel="Proporción minoritaria")

    scatter_columna(axes[4], df, "entropia", "proporcion_min",
                    "Entropía vs Proporción minoritaria",
                    xlabel="Entropía", ylabel="Proporción minoritaria")

    scatter_columna(axes[5], df, "densidad", "vecinos_validos_por_percentil",
                    "Densidad vs Vecinos válidos",
                    xlabel="Densidad", ylabel="Vecinos válidos")

    fig.suptitle(
        "Relaciones internas entre métricas – " + titulo_base,
        fontsize=13
    )
    fig.tight_layout(rect=[0, 0, 1, 0.93])

    fname = f"{meta['base_tag']}_fig3_relaciones.png"
    path = output_dir / fname
    fig.savefig(path, dpi=150)
    plt.close(fig)
    return fname


# ---------------------------------------------------------------------
# FIGURA 4 – RIESGO vs ENTROPÍA coloreado por cantidad de sintéticos
# ---------------------------------------------------------------------
def generar_figura_riesgo_entropia_color(df, meta, output_dir: Path) -> str:
    """
    Pregunta: ¿las semillas con mayor riesgo y entropía generan más sintéticos?

    - eje X: riesgo
    - eje Y: entropía
    - color: cantidad de sintéticos generados por esa semilla
    """
    if "riesgo" not in df.columns or "entropia" not in df.columns:
        return None

    titulo_base = (
        f"{meta['dataset']}  D={meta['D']}  R={meta['R']}  "
        f"P={meta['P']}  I={meta['I']}"
    )

    x = df["riesgo"].values
    y = df["entropia"].values
    c = df.get("synthetics_from_this_seed", pd.Series(np.zeros(len(df)))).values

    fig, ax = plt.subplots(figsize=(8, 6))

    sc = ax.scatter(x, y, c=c, s=25, alpha=0.8)
    ax.set_xlabel("Riesgo (proporción de vecinos mayoritarios)")
    ax.set_ylabel("Entropía del vecindario")
    ax.set_title("Riesgo vs Entropía coloreado por sintéticos\n" + titulo_base)

    cbar = fig.colorbar(sc, ax=ax)
    cbar.set_label("Sintéticos generados por semilla")

    fig.tight_layout()

    fname = f"{meta['base_tag']}_fig4_riesgo_entropia_color.png"
    path = output_dir / fname
    fig.savefig(path, dpi=150)
    plt.close(fig)
    return fname


# ---------------------------------------------------------------------
# FIGURA 5 – SINTÉTICOS EXTREMOS (outliers de generación)
# ---------------------------------------------------------------------
def generar_figura_sinteticos_extremos(df, meta, output_dir: Path) -> str:
    """
    Pregunta: ¿qué tan extremos son los valores de sintéticos por semilla?
    - Marca percentiles 10 y 90 sobre el histograma
    - Muestra un scatter Riesgo vs Sintéticos resaltando las semillas extremas
    """
    if "synthetics_from_this_seed" not in df.columns:
        return None

    titulo_base = (
        f"{meta['dataset']}  D={meta['D']}  R={meta['R']}  "
        f"P={meta['P']}  I={meta['I']}"
    )

    serie = df["synthetics_from_this_seed"].dropna().values
    if len(serie) == 0:
        return None

    p10, p90 = np.percentile(serie, [10, 90])

    fig, axes = plt.subplots(1, 2, figsize=(14, 5))

    # Histograma con cortes 10% y 90%
    axes[0].hist(serie, bins=30)
    axes[0].axvline(p10, color="red", linestyle="--", label=f"P10={p10:.1f}")
    axes[0].axvline(p90, color="green", linestyle="--", label=f"P90={p90:.1f}")
    axes[0].set_title("Distribución de sintéticos por semilla")
    axes[0].set_xlabel("Sintéticos generados")
    axes[0].set_ylabel("Número de semillas")
    axes[0].legend()

    # Scatter Riesgo vs Sintéticos, coloreando extremos
    if "riesgo" in df.columns:
        riesgo = df["riesgo"].values
        sint = df["synthetics_from_this_seed"].values

        mask_bajo = sint <= p10
        mask_alto = sint >= p90
        mask_medio = (~mask_bajo) & (~mask_alto)

        axes[1].scatter(riesgo[mask_medio], sint[mask_medio],
                        s=20, alpha=0.4, label="zona media")
        axes[1].scatter(riesgo[mask_bajo], sint[mask_bajo],
                        s=40, alpha=0.8, label="extremos bajos")
        axes[1].scatter(riesgo[mask_alto], sint[mask_alto],
                        s=40, alpha=0.8, label="extremos altos")

        axes[1].set_xlabel("Riesgo")
        axes[1].set_ylabel("Sintéticos generados")
        axes[1].set_title("Sintéticos extremos vs Riesgo")
        axes[1].legend()

    fig.suptitle("Análisis de sintéticos extremos – " + titulo_base, fontsize=13)
    fig.tight_layout(rect=[0, 0, 1, 0.93])

    fname = f"{meta['base_tag']}_fig5_sinteticos_extremos.png"
    path = output_dir / fname
    fig.savefig(path, dpi=150)
    plt.close(fig)
    return fname


# ---------------------------------------------------------------------
# FIGURA 6 – “FRONTERA IDEAL” segun percentiles de riesgo/entropía/densidad
# ---------------------------------------------------------------------
def generar_figura_frontera_ideal(df, meta, output_dir: Path) -> str:
    """
    Pregunta: ¿qué semillas caen en la 'frontera ideal' según tus métricas?

    Definición pragmática:
      - riesgo en [P25_riesgo, P75_riesgo]
      - entropía en [P25_entropia, P75_entropia]
      - densidad en [P25_densidad, P75_densidad]

    Se remarcan esas semillas sobre el plano Riesgo–Entropía.
    """
    columnas_necesarias = ["riesgo", "entropia", "densidad"]
    if not all(col in df.columns for col in columnas_necesarias):
        return None

    titulo_base = (
        f"{meta['dataset']}  D={meta['D']}  R={meta['R']}  "
        f"P={meta['P']}  I={meta['I']}"
    )

    riesgo = df["riesgo"].values
    ent = df["entropia"].values
    dens = df["densidad"].values

    if len(riesgo) == 0:
        return None

    r25, r75 = np.percentile(riesgo, [25, 75])
    e25, e75 = np.percentile(ent, [25, 75])
    d25, d75 = np.percentile(dens, [25, 75])

    # máscara frontera ideal
    mask_frontera = (
        (riesgo >= r25) & (riesgo <= r75) &
        (ent >= e25) & (ent <= e75) &
        (dens >= d25) & (dens <= d75)
    )

    fig, ax = plt.subplots(figsize=(8, 6))

    # resto de semillas
    ax.scatter(riesgo[~mask_frontera], ent[~mask_frontera],
               s=15, alpha=0.3, label="Otras semillas")

    # semillas en frontera ideal
    ax.scatter(riesgo[mask_frontera], ent[mask_frontera],
               s=40, alpha=0.9, label="Frontera ideal (según percentiles)")

    ax.set_xlabel("Riesgo")
    ax.set_ylabel("Entropía")
    ax.set_title(
        "Riesgo vs Entropía – Frontera ideal según percentiles\n" + titulo_base
    )
    ax.legend()

    # información de percentiles en un texto
    texto = (
        f"Riesgo P25–P75: [{r25:.2f}, {r75:.2f}]\n"
        f"Entropía P25–P75: [{e25:.2f}, {e75:.2f}]\n"
        f"Densidad P25–P75: [{d25:.2f}, {d75:.2f}]"
    )
    ax.text(0.05, 0.95, texto, transform=ax.transAxes,
            fontsize=9, va="top",
            bbox=dict(boxstyle="round", facecolor="white", alpha=0.7))

    fig.tight_layout()

    fname = f"{meta['base_tag']}_fig6_frontera_ideal.png"
    path = output_dir / fname
    fig.savefig(path, dpi=150)
    plt.close(fig)
    return fname


In [ ]:
registros_figuras = []

for item in logs:
    df = item["df"]
    meta = item["meta"]

    print(f"🧪 Generando figuras para: {meta['nombre_archivo']}")

    figuras = []

    # --- Figura 1: distribuciones ---
    f1 = generar_figura_distribuciones(df, meta, IMG_DIR)
    figuras.append(f1)

    # ---------------------------------------------------------
    # --- FIGURA 2: el grupo de 4 gráficos en 1 SOLO PNG ---
    # ---------------------------------------------------------
    # Extraer las series necesarias (adaptá nombres a tus columnas reales)
    riesgos = df["riesgo"].values
    entropias = df["entropia"].values
    sinteticos_por_semilla = df["synthetics_from_this_seed"].values

    # percentiles que ya usabas
    p10 = np.percentile(sinteticos_por_semilla, 10)
    p90 = np.percentile(sinteticos_por_semilla, 90)

    # filtrar extremos (ajustá si tus columnas tienen otros nombres)
    riesgos_bajos = riesgos[sinteticos_por_semilla <= p10]
    sinteticos_bajos = sinteticos_por_semilla[sinteticos_por_semilla <= p10]

    riesgos_altos = riesgos[sinteticos_por_semilla >= p90]
    sinteticos_altos = sinteticos_por_semilla[sinteticos_por_semilla >= p90]

    # si tenés frontera ideal, agregala; si no, omitila
    riesgos_ideales = df.get("riesgo_ideal", riesgos)
    entropias_ideales = df.get("entropia_ideal", entropias)

    # --- Crear la figura 4-en-1 ---
    fig, axs = plt.subplots(2, 2, figsize=(16, 10))
    (ax1, ax2), (ax3, ax4) = axs

    # 1) Riesgo vs Entropía – Frontera ideal
    ax1.scatter(riesgos, entropias, color="steelblue", alpha=0.6)
    ax1.scatter(riesgos_ideales, entropias_ideales, color="darkorange", alpha=0.9)
    ax1.set_title("Riesgo vs Entropía – Frontera ideal")
    ax1.set_xlabel("Riesgo")
    ax1.set_ylabel("Entropía")

    # 2) Histograma de sintéticos
    ax2.hist(sinteticos_por_semilla, bins=20, color="steelblue", alpha=0.8)
    ax2.axvline(p10, color="red", linestyle="--")
    ax2.axvline(p90, color="green", linestyle="--")
    ax2.set_title("Distribución de sintéticos por semilla")
    ax2.set_xlabel("Sintéticos generados")
    ax2.set_ylabel("N° semillas")

    # 3) Sintéticos extremos vs Riesgo
    ax3.scatter(riesgos, sinteticos_por_semilla, color="steelblue", alpha=0.5, label="zona media")
    ax3.scatter(riesgos_bajos, sinteticos_bajos, color="orange", label="extremos bajos")
    ax3.scatter(riesgos_altos, sinteticos_altos, color="green", label="extremos altos")
    ax3.set_title("Sintéticos extremos vs Riesgo")
    ax3.set_xlabel("Riesgo")
    ax3.set_ylabel("Sintéticos generados")
    ax3.legend()

    # 4) Riesgo vs Entropía coloreado por sintéticos
    sc = ax4.scatter(riesgos, entropias, c=sinteticos_por_semilla, cmap="viridis")
    ax4.set_title("Riesgo vs Entropía coloreado por sintéticos")
    ax4.set_xlabel("Riesgo")
    ax4.set_ylabel("Entropía")
    fig.colorbar(sc, ax=ax4, label="Sintéticos por semilla")

    plt.tight_layout()

    # guardar PNG
    f_4en1 = f"{meta['base_tag']}_figura_4_en_1.png"
    path_png = IMG_DIR / f_4en1
    fig.savefig(path_png, dpi=150)
    plt.close(fig)

    figuras.append(f_4en1)
    # ---------------------------------------------------------

    registros_figuras.append({
        "meta": meta,
        "figuras": figuras,
    })

print("\nTotal configuraciones con figuras generadas:", len(registros_figuras))


In [ ]:
index_path = OUTPUT_DIR / "index.html"

html_inicio = """<!DOCTYPE html>
<html lang="es">
<head>
<meta charset="utf-8">
<title>Reporte PCSMOTE - Logs por muestra</title>
<style>
body {
    font-family: Arial, sans-serif;
    margin: 20px;
}
h1 {
    text-align: center;
}
.section {
    margin-bottom: 40px;
    border-bottom: 1px solid #ccc;
    padding-bottom: 20px;
}
.meta {
    font-size: 0.9rem;
    color: #555;
    margin-bottom: 10px;
}
.grid-img {
    display: flex;
    flex-wrap: wrap;
    gap: 20px;
}
.grid-img img {
    max-width: 480px;
    border: 1px solid #ddd;
}
</style>
</head>
<body>
<h1>Reporte de análisis por muestra – PCSMOTE</h1>
"""

html_fin = """
</body>
</html>
"""

bloques = []

for reg in registros_figuras:
    meta = reg["meta"]
    figs = reg["figuras"]

    titulo = (
        f"{meta['dataset']} – D={meta['D']}  R={meta['R']}  "
        f"P={meta['P']}  I={meta['I']}"
    )

    bloque = []
    bloque.append('<div class="section">')
    bloque.append(f"<h2>{titulo}</h2>")
    bloque.append(
        f'<div class="meta">Archivo origen: {meta["nombre_archivo"]}</div>'
    )
    bloque.append('<div class="grid-img">')

    for fname in figs:
        src = f"img/{fname}"
        bloque.append(f'<div><img src="{src}" alt="{fname}"></div>')

    bloque.append("</div>")  # grid-img
    bloque.append("</div>")  # section

    bloques.append("\n".join(bloque))

html_contenido = html_inicio + "\n".join(bloques) + html_fin

with open(index_path, "w", encoding="utf-8") as fh:
    fh.write(html_contenido)

print("✅ index.html generado en:", index_path)
print("Abrir en el navegador para ver todas las figuras.")
